In [139]:
import time
import os 
from dotenv import load_dotenv

import spotipy
from spotipy.oauth2 import SpotifyOAuth

import pprint

import pandas as pd

In [34]:
def get_spotify_client(client_id, client_secret, redirect_uri, scope):
    return spotipy.Spotify(auth_manager=SpotifyOAuth(
        client_id=client_id, 
        client_secret=client_secret, 
        scope=scope,
        redirect_uri=redirect_uri
    ))

In [35]:
load_dotenv()
client_id = os.environ['CLIENT_ID']
client_secret = os.environ['CLIENT_SECRET']
redirect_uri = 'http://localhost:3000'
scope = "user-library-read user-top-read"

sp = get_spotify_client(client_id, client_secret, redirect_uri, scope)

In [89]:
def get_top_stats(client, limit=20, offset=0, time_range='medium_term', top_type='artists'):
    if top_type == 'artists':
        return client.current_user_top_artists(limit, offset, time_range)
    elif top_type == 'tracks':
        return client.current_user_top_tracks(limit, offset, time_range)

In [162]:
def get_top_artists(client, limit=90, time_range='long_term'):
    if limit <= 50:
        items = get_top_stats(client, limit=limit, time_range=time_range, top_type='artists')['items'] 
    else:
        items = get_top_stats(client, limit=40, time_range=time_range, top_type='artists')['items'] 
        items += get_top_stats(client, limit=limit-40, offset=40, time_range=time_range, top_type='artists')['items'] 
    return pd.DataFrame(items, columns=['name','genres','popularity'])

In [160]:
artists_df = get_top_artists(sp)

In [161]:
artists_df.to_csv('top90artists.csv')